# Supplementary - Whole-brain gradients and eigenmodes analysis

Related to Supplementary Figure S11.

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *

from scipy.stats import zscore, spearmanr
from scipy.stats import percentileofscore
from brainsmash.mapgen.base import Base

# Loading whole-brain data

In [ ]:
# Loading whole-brain functional data
nodes = np.load('../Files/PaperNetworks/centroids_hires.npy')
FC = np.load('../Files/PaperNetworks/FC_hires_spontaneous.npy')
FC_stim = np.load('../Files/PaperNetworks/FC_hires_stim.npy')

# Loading whole-brain eigenmodes
eigenmodes = np.load('../Files/zebrafish_wholebrain_eigenmodes_single.npy')
vertices = np.load('../Files/zebrafish_wholebrain_vertices_single.npy')

In [ ]:
plt.imshow(FC)

# Correlation vs distance relationship

In [ ]:
d = compute_distances(nodes, nodes)
R = FC.shape[0]
d = (1 / 3) * (d[:R, :R] + d[R:, R:] + d[:R, R:])

In [ ]:
%matplotlib inline

In [ ]:
triangle = np.triu_indices(FC.shape[0], 1)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
ax.scatter(d[triangle], FC[triangle], alpha=0.1, color='black', edgecolor=None, s=1, rasterized=True)
ax.set_xlabel('Distance (microns)')
ax.set_ylabel('Correlation')
ax.spines[['top', 'right']].set_visible(False)


In [ ]:
spearmanr(d[triangle], FC[triangle])

In [ ]:
d_values = d[triangle]
FC_values = FC[triangle]

# Gradients & eigenmodes

In [ ]:
N_modes = FC.shape[0]
gradients, _ = diffusion_mapping(np.abs(FC), n_components=N_modes)

In [ ]:
d = compute_distances(vertices, nodes[:FC.shape[0]])
eigenmodes_ = eigenmodes[:, np.argmin(d, axis=0)]

In [ ]:
mode_similarity, mapping = compute_mode_similarity_matrix(eigenmodes_[1:31], gradients.T[:30], return_mapping=True)

In [ ]:
plt.figure(figsize=(5, 5), dpi=150)

plt.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0.1, vmax=0.75)
plt.ylabel('Geometric modes')
plt.xlabel('Functional gradients')

# Spatial statistics

In [ ]:
def shuffle_vector_brainsmash(vector, distances, resample=True, n_iters=1):
    base = Base(x=vector, D=distances, resample=resample)
    surrogate = base(n=n_iters)
    return surrogate

def shuffle_vectors_brainsmash(vectors, distances, resample=True):
    shuffled_vectors = []
    for v in vectors:
        base = Base(x=v, D=distances, resample=resample)
        surrogate = base(n=1)
        shuffled_vectors.append(surrogate)
    return np.array(shuffled_vectors)

In [ ]:
empirical_score = np.mean(np.abs(np.diag(mode_similarity))[:30])
print(empirical_score)

In [ ]:
modes1, modes2 = eigenmodes_[1:31], gradients.T[:30]
d = compute_distances(nodes[:FC.shape[0]], nodes[:FC.shape[0]])

In [ ]:
null_similarity_matrices = []

for _ in tqdm(range(1000)):
    
    shuffled_modes1 = shuffle_vectors_brainsmash(modes1, d, resample=False)
    corrs_null = compute_mode_similarity_matrix(shuffled_modes1, modes2, return_mapping=False)
    null_similarity_matrices.append(corrs_null)

null_similarity_matrices = np.stack(null_similarity_matrices, axis=0)

In [ ]:
np.save('../Results/null_similarity_wholebrain.npy', null_similarity_matrices)

#### Loading and computing stats

In [ ]:
null_similarity_matrices = np.load('../Results/null_similarity_wholebrain.npy')

In [ ]:
plt.figure(figsize=(5, 5), dpi=150)

plt.imshow(np.abs(null_similarity_matrices[5]), cmap='Reds', vmin=0.1, vmax=0.75)
plt.ylabel('Geometric modes')
plt.xlabel('Functional gradients')

In [ ]:
null_scores = []
for m in null_similarity_matrices:
    null_scores.append(np.mean(np.abs(np.diag(m))[:30]))

In [ ]:
percentileofscore(null_scores, empirical_score)

# Plotting eigenmodes and gradients

Pre-rendering the 3D scatter plots of whole-brain eigenmodes and gradients in RGB arrays to facilitate the multipanel figure later.

In [ ]:
i = 0
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(2, 2), dpi=600)
ax.scatter(nodes[:, 0], nodes[:, 1], -nodes[:, 2], c=np.sign(mode_similarity[i, i]) * double(gradients[:, mapping[i]]), alpha=0.75, cmap='coolwarm', s=2, edgecolor='None')
ax.set_axis_off()
#ax.set_ylim(ax.get_ylim()[::-1])
#ax.set_zlim(ax.get_zlim()[::-1])
ax.set_xlim([0, 1000])
ax.set_ylim([0, 1000])
ax.set_zlim([0, 1000])
ax.view_init(elev=40, azim=-140)
plt.show()

    

In [ ]:
figures_gradients = []

for i in range(30):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(2, 2), dpi=600)
    ax.scatter(nodes[:, 0], nodes[:, 1], -nodes[:, 2], c=np.sign(mode_similarity[i, i]) * double(gradients[:, mapping[i]]), alpha=0.75, cmap='coolwarm', s=2, edgecolor='None')
    ax.set_axis_off()
    #ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_zlim(ax.get_zlim()[::-1])
    ax.set_xlim([0, 1000])
    ax.set_ylim([0, 1000])
    ax.set_zlim([0, 1000])
    ax.view_init(elev=40, azim=-140)
    #plt.show()
    
    fig_array = figure_to_array(fig)
    figures_gradients.append(fig_array[720:1050, 375:790])
    plt.close()

In [ ]:
plt.imshow(figures_gradients[0])

In [ ]:
figures_eigenmodes = []

for i in range(10):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(2, 2), dpi=600)
    ax.scatter(nodes[:, 0], nodes[:, 1], -nodes[:, 2], c=double(eigenmodes_[i+1]), alpha=0.75, cmap='coolwarm', s=2, edgecolor='None')
    ax.set_axis_off()
    #ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_zlim(ax.get_zlim()[::-1])
    ax.set_xlim([0, 1000])
    ax.set_ylim([0, 1000])
    ax.set_zlim([0, 1000])
    ax.view_init(elev=40, azim=-140)
    #plt.show()

    
    fig_array = figure_to_array(fig)
    figures_eigenmodes.append(fig_array[720:1050, 375:790])
    plt.close()

In [ ]:
plt.imshow(figures_eigenmodes[0])

# Rendering figure

In [ ]:
density = compute_scatter_density(d_values, FC_values)

In [ ]:
fig = PaperFigure(figsize=(7, 4.5), dpi=300)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding axes ----------------------------------------------------------------------------

w = 1.5
fig.add_axes('correlations', (3, 0), w, w)
fig.add_axes('scatter', (5.5, 0), w, w)

N_modes = 7
w = 5 / N_modes
for i in range(N_modes):
    fig.add_axes(f'eigenmode{i}', (i * w, 2), w, w)
    fig.add_axes(f'gradient{i}', (i * w, 2 + w), w, w)

fig.add_axes('mapping', (5.5, 2), 1.5, 1.5)

fig.set_line_thickness(0.5)

# Filling content ------------------------------------------------------------------------

ax = fig.axes['correlations']
ax.imshow(FC, cmap='coolwarm', vmin=-0.5, vmax=0.5)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['scatter']
#ax.scatter(d_values, FC_values, s=0.25, edgecolor=None, alpha=0.1, color='black')
o = np.argsort(density)
v = np.max(density)
ax.scatter(d_values[o], FC_values[o], edgecolor=None, c=density[o], cmap='coolwarm', s=1, alpha=0.1, vmin=-1.25 * v, vmax=v, rasterized=True)
#ax.set_xticks([0, 275])
ax.set_xlim([0, 800])
ax.set_ylim([-0.25, 1.0])
#ax.set_yticks([0, 0.6])
ax.spines[['top', 'right']].set_visible(False)

for i in range(N_modes):
    ax = fig.axes[f'eigenmode{i}']
    ax.imshow(figures_eigenmodes[i], aspect='auto')
    ax.axis('off')

for i in range(N_modes):
    ax = fig.axes[f'gradient{i}']
    ax.imshow(figures_gradients[mapping[i]], aspect='auto')
    ax.axis('off')

ax = fig.axes['mapping']
ax.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0.1, vmax=0.75)
ax.set_xticks([])
ax.set_yticks([])

fig.show()

In [ ]:
fig.save('../Figures/supp_wholebrain_incomplete.svg')

#### Manually annotated values

Ordering of gradients

In [ ]:
print(mapping)